# Neuron Classification Using Neural Networks in Julia

This project demonstrates how to classify neurons based on their firing patterns using a simple neural network implemented in Julia. It covers various aspects of data science, machine learning, and computational neuroscience.

## Step 1 - Generate Synthetic Data

Start by generating a synthetic dataset of neurons. Each neuron has a unique ID, a firing rate (how often it sends a signal), and a type (either excitatory or inhibitory).

In [1]:
using Random, CSV, DataFrames

function generate_data(n::Int)
    data = DataFrame(id = 1:n, firing_rate = rand(50:150, n), class = rand(["Excitatory", "Inhibitory"], n))
    CSV.write("neuron_data.csv", data)
end

# Generate data
generate_data(1000)

"neuron_data.csv"

## Step 2 - Load and Explore the Data

Next, the generated data loads from the CSV file and displays the first few rows to understand its structure. Then the data is explored by summarizing its statistics.

In [2]:
using CSV, DataFrames

# Load the data and display the first 5 rows
data = CSV.read("neuron_data.csv", DataFrame)
println(first(data, 5))

# Explore the data
println(describe(data))

5×3 DataFrame
 Row │ id     firing_rate  class      
     │ Int64  Int64        String15   
─────┼────────────────────────────────
   1 │     1          147  Excitatory
   2 │     2          126  Inhibitory
   3 │     3          106  Inhibitory
   4 │     4           84  Excitatory
   5 │     5           90  Excitatory
3×7 DataFrame
 Row │ variable     mean    min         median  max         nmissing  eltype   
     │ Symbol       Union…  Any         Union…  Any         Int64     DataType 
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ id           500.5   1           500.5   1000               0  Int64
   2 │ firing_rate  99.246  50          99.0    150                0  Int64
   3 │ class                Excitatory          Inhibitory         0  String15


## Step 3 - Data Processing

The data needs to be processed to make it suitable for training a neural network. This involves converting the neuron types into a format that the neural network can understand and splitting the data into training and testing sets.

In [3]:
using MLJ

# Encode categorical variables
data.class = coerce(data.class, Multiclass)

# Split the data
using MLJBase

train, test = partition(eachindex(data.id), 0.8, shuffle=true)
train_data = data[train, :]
test_data = data[test, :]

Row,id,firing_rate,class
,Int64,Int64,Cat…
1,649,109,Excitatory
2,849,129,Inhibitory
3,855,101,Inhibitory
4,467,60,Excitatory
5,105,66,Inhibitory
6,810,55,Excitatory
7,326,51,Inhibitory
8,287,113,Excitatory
9,179,61,Excitatory


## Step 4 - Define the Neural Network Model

Define a simple neural network using the Flux library. This network will learn to classify neurons based on their firing rates.


In [4]:
using Flux
using Optimisers  # Import optimisers for ADAM

model = Chain(
    Dense(1, 10, relu),
    Dense(10, 2),
    softmax
)

Chain(
  Dense(1 => 10, relu),                 # 20 parameters
  Dense(10 => 2),                       # 22 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 42 parameters, 376 bytes.

## Step 5 - Train the Neural Network

Prepare the training data and train the neural network. The network adjusts its internal settings to learn how to classify neurons.


In [5]:
# Prepare data for training
X_train = Matrix(train_data[:, :firing_rate]')
y_train = Flux.onehotbatch(train_data.class, ["Excitatory", "Inhibitory"])

# Define the loss function and optimizer
loss(model, x, y) = Flux.crossentropy(model(x), y)
opt = Flux.setup(Optimisers.ADAM(), model)

# Training loop
data = [(X_train, y_train)]
parameters = Flux.params(model)
for epoch in 1:100
    Flux.train!(loss, params(model), data, opt)
end

# Print the trained model
println("Trained Model:")
println(model)

Trained Model:
Chain(Dense(1 => 10, relu), Dense(10 => 2), softmax)


## Step 6 - Evaluate the Neural Network

Evaluate the trained neural network using the test data. This involves making predictions and calculating the accuracy of the model.


In [6]:
# Prepare data for testing
X_test = Matrix(test_data[:, :firing_rate]')
y_test = test_data.class

# Make predictions
y_pred = model(X_test)
y_pred_labels = Flux.onecold(y_pred, ["Excitatory", "Inhibitory"])

# Calculate accuracy
accuracy = mean(y_pred_labels .== y_test)
println("Model Accuracy: ", accuracy)

Model Accuracy: 0.53


## Step 7 - Visualize the Data

In [ ]:
# Visualize the data
using Plots
pyplot()

# Plot the data
scatter(data.id, data.firing_rate, group=data.class, legend=:top, title="Neuron Firing Rates", xlabel="Neuron ID", ylabel="Firing Rate")

# Save the first plot
savefig("neuron_firing_rates.png")

# Add predictions to the test data
test_data.predicted_class = y_pred_labels

# Plot the actual vs predicted classifications
p = scatter(test_data.id, test_data.firing_rate, group=test_data.class, legend=:topright, title="Actual vs Predicted Classifications", xlabel="Neuron ID", ylabel="Firing Rate", label="Actual")
scatter!(p, test_data.id, test_data.firing_rate, group=test_data.predicted_class, markershape=:cross, label="Predicted")

# Save the second plot
savefig(p, "actual_vs_predicted.png")

# Display the plots
display(p)

ERROR: LoadError: PyCall not properly installed. Please run Pkg.build("PyCall")
Stacktrace:
 [1] error(s::String)
   @ Base .\error.jl:35
 [2] top-level scope
   @ C:\Users\ryan.hopkins\.julia\packages\PyCall\1gn3u\src\startup.jl:44
 [3] include(mod::Module, _path::String)
   @ Base .\Base.jl:557
 [4] include(x::String)
   @ PyCall C:\Users\ryan.hopkins\.julia\packages\PyCall\1gn3u\src\PyCall.jl:1
 [5] top-level scope
   @ C:\Users\ryan.hopkins\.julia\packages\PyCall\1gn3u\src\PyCall.jl:38
 [6] include
   @ .\Base.jl:557 [inlined]
 [7] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::String)
   @ Base .\loading.jl:2881
 [8] top-level scope
   @ stdin:6
in expression starting at C:\Users\ryan.hopkins\.julia\packages\PyCall\1gn3u\src\startup.jl:41
in expression starting at C:\Users\ryan.hopkins\.julia\packages\PyCall\1gn3u\src\PyCall.jl:

ErrorException: Failed to precompile PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee] to "C:\\Users\\ryan.hopkins\\.julia\\compiled\\v1.11\\PyPlot\\jl_A135.tmp".

## Conclusion

This project demonstrates how to use Julia and Flux to build a neural network for classifying neurons based on their firing patterns. The final output is the accuracy of the model, which indicates how well the network can classify new neurons.
